### Simple GEN-AI app using Langchain

Load all the environment variables

In [26]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")

Scrape data from a website

In [27]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

class_label = "beebom-single-content-container"
web_url = "https://beebom.com/valorant-characters-agents-abilities"

loader = WebBaseLoader(web_paths=[web_url],
                bs_kwargs={
                    "parse_only": bs4.SoupStrainer(class_=class_label),
                })
docs = loader.load()

Divide the raw data from the website into chunks

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

splitted_docs = text_splitter.split_documents(docs)

Generate embeddings of the documents

In [29]:
from langchain_huggingface import HuggingFaceEmbeddings

EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
embedder = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    multi_process=True,
    model_kwargs={
        "device": "cuda",
    },
    encode_kwargs={
        "normalize_embeddings": True,
    }
)

Store the embeddings in the vector database FAISS

In [30]:
from langchain_community.vectorstores import FAISS

vectorstore_db = FAISS.from_documents(
    splitted_docs,
    embedder
)

Query the vectorstore DB

In [31]:
query = "What are the abilities of Killjoy?"
result = vectorstore_db.similarity_search(query, k=3)

Add retrieval chain to the LLM

In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant. Use the following pieces of context wrapped within <context>..</context> to answer the question.
    <context>
    {context}
    </context>
    Question: {input}
    """
)

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

retriever = vectorstore_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
    }
)

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt)


retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [41]:
response = retrieval_chain.invoke({"input": query})

In [44]:
response['answer']

"According to the context, Killjoy's abilities are:\n\n1. Alarmbot (Q) - Puts a scanner bot in place that chases down an enemy and explodes on contact, making the affected enemy vulnerable for a brief duration.\n2. Nanoswarm (C) - Places swarm grenades that create an energy field and cause damage to anyone inside the field when activated.\n3. Turret (E) (Signature) - Puts a sentry gun in place that scans and inflicts damage on enemies within its 100-degree forward vision.\n4. Lockdown (X) (Ultimate) - Initiates a lockdown by putting down a device that creates a massive energy dome, disarming and slowing down all enemies within the area for a brief duration."